In [1]:
import requests
from lxml import etree
import lxml
from lxml.html import fromstring
from io import StringIO, BytesIO
import pandas as pd
import threading

In [2]:
url = "https://www.vultr.com/faq/#downloadspeedtests"
r = requests.get(url)
if r.status_code != 200:
    raise Exception("vultr server hosts list not available")
html = r.text

In [3]:
# tree = lxml.html.fromstring(html)
tree = fromstring(html)
table = tree.xpath('//table[@class="table speedtest_table"]')[0]

In [4]:
link_dict = dict()
for tr in table.find("tbody").findall("tr"):
    tds = tr.findall("td")
    name = tds[0].text_content().strip()
    page_link = tds[1].find("a").attrib["href"]
    link_dict[name] = page_link

In [5]:
info_list = []

In [6]:
class GetInfoThread(threading.Thread):
    def __init__(self, name, link):
        threading.Thread.__init__(self)
        self.name = name
        self.link = link
        
    def run(self):
        try:
            sub_r = requests.get(self.link)
            if sub_r.status_code != 200:
                raise Exception("cannot get test info of {}".format(name))
            sub_html = sub_r.text
            sub_tree = lxml.html.fromstring(sub_html)
            addr = sub_tree.xpath('//a[@id="useripv4"]')[0].text_content().strip()
            file = sub_tree.xpath('//a[@id="testfile"]')[0].attrib["href"].strip()
            file = self.link + file
            info_list.append({"name": str(self.name), "addr": str(addr), "test_file": str(file)})
        except Exception as e:
            print(e)

In [7]:
threads = []
for name, link in link_dict.items():
    t = GetInfoThread(name, link);
    threads.append(t)
    t.start()
for t in threads:
    t.join()

In [8]:
pd.DataFrame(info_list)

,addr,name,test_file
0,108.61.149.182,New York (NJ),https://nj-us-ping.vultr.com/vultr.com.100MB.bin
1,108.61.193.166,"Atlanta, Georgia",https://ga-us-ping.vultr.com/vultr.com.100MB.bin
2,149.248.50.81,"Toronto, Canada",https://tor-ca-ping.vultr.com/vultr.com.100MB.bin
3,108.61.224.175,"Dallas, Texas",https://tx-us-ping.vultr.com/vultr.com.100MB.bin
4,107.191.51.12,"Chicago, Illinois",https://il-us-ping.vultr.com/vultr.com.100MB.bin
5,108.61.194.105,"Seattle, Washington",https://wa-us-ping.vultr.com/vultr.com.100MB.bin
6,104.156.244.232,"Miami, Florida",https://fl-us-ping.vultr.com/vultr.com.100MB.bin
7,108.61.209.127,"Paris, France",https://par-fr-ping.vultr.com/vultr.com.100MB.bin
8,108.61.198.102,"Amsterdam, NL",https://ams-nl-ping.vultr.com/vultr.com.100MB.bin
9,108.61.210.117,"Frankfurt, DE",https://fra-de-ping.vultr.com/vultr.com.100MB.bin


In [9]:
df = pd.DataFrame(info_list).set_index("name")

In [10]:
df.to_csv("./vultr.csv")